In [102]:
#import libraries
import pandas as pd
import cv2 
import numpy as np
import sklearn
import os

In [117]:
import tensorflow as tf

In [118]:
train_dir = "train"
test_dir = "test"

In [119]:
dataGenerator = tf.keras.preprocessing.image.ImageDataGenerator(rescale=1./255, horizontal_flip=True, validation_split=0.2)

training_data = dataGenerator.flow_from_directory(train_dir, batch_size=64, target_size=(48, 48), shuffle=True, color_mode='grayscale', class_mode='categorical', subset='training')
validation_set = dataGenerator.flow_from_directory(train_dir, batch_size=64, target_size=(48, 48), shuffle=True, color_mode='grayscale', class_mode='categorical', subset='validation')

testDataGenerator = tf.keras.preprocessing.image.ImageDataGenerator(rescale=1./255, horizontal_flip=True)

test_data = testDataGenerator.flow_from_directory(test_dir, batch_size=64, target_size=(48, 48), shuffle=True, color_mode='grayscale', class_mode='categorical')

Found 22968 images belonging to 7 classes.
Found 5741 images belonging to 7 classes.
Found 7178 images belonging to 7 classes.


In [120]:
def create_model():
    weight_decay = 1e-4
    model = tf.keras.models.Sequential()

    model.add(tf.keras.layers.Conv2D(64, (4, 4), padding='same', kernel_regularizer=tf.keras.regularizers.l2(weight_decay), input_shape=(48, 48, 1)))
    model.add(tf.keras.layers.Activation('relu'))
    model.add(tf.keras.layers.BatchNormalization())
    model.add(tf.keras.layers.Conv2D(64, (4, 4), padding='same', kernel_regularizer=tf.keras.regularizers.l2(weight_decay)))
    model.add(tf.keras.layers.Activation('relu'))
    model.add(tf.keras.layers.BatchNormalization())
    model.add(tf.keras.layers.MaxPool2D(pool_size=(2, 2)))
    model.add(tf.keras.layers.Dropout(0.2))

    model.add(tf.keras.layers.Conv2D(128, (4, 4), padding='same', kernel_regularizer=tf.keras.regularizers.l2(weight_decay)))
    model.add(tf.keras.layers.Activation('relu'))
    model.add(tf.keras.layers.BatchNormalization())
    model.add(tf.keras.layers.MaxPool2D(pool_size=(2, 2)))
    model.add(tf.keras.layers.Dropout(0.3))
    
    model.add(tf.keras.layers.Conv2D(128, (4, 4), padding='same', kernel_regularizer=tf.keras.regularizers.l2(weight_decay)))
    model.add(tf.keras.layers.Activation('relu'))
    model.add(tf.keras.layers.BatchNormalization())
    model.add(tf.keras.layers.Conv2D(128, (4, 4), padding='same', kernel_regularizer=tf.keras.regularizers.l2(weight_decay)))
    model.add(tf.keras.layers.Activation('relu'))
    model.add(tf.keras.layers.BatchNormalization())
    model.add(tf.keras.layers.MaxPool2D(pool_size=(2, 2)))
    model.add(tf.keras.layers.Dropout(0.4))
    model.add(tf.keras.layers.Flatten())
    model.add(tf.keras.layers.Dense(128, activation="linear"))
    model.add(tf.keras.layers.Activation('relu'))
    model.add(tf.keras.layers.Dense(7, activation='softmax'))
    
    return model

In [121]:
model = create_model()

model.compile(loss='categorical_crossentropy', optimizer=tf.keras.optimizers.Adam(0.0003), metrics=['accuracy'])

In [122]:
checkpointer = [tf.keras.callbacks.EarlyStopping(monitor = 'val_accuracy', verbose = 1, restore_best_weights=True, mode="max",patience = 10),
                tf.keras.callbacks.ModelCheckpoint(
                    filepath='final_model_weights.hdf5',
                    monitor="val_accuracy",
                    verbose=1,
                    save_best_only=True,
                    mode="max")]

In [125]:
steps_per_epoch = training_data.n // training_data.batch_size
validation_steps = validation_set.n // validation_set.batch_size

history = model.fit(x=training_data,
                 validation_data=validation_set,
                 epochs=90,
                 callbacks=[checkpointer],
                 steps_per_epoch=steps_per_epoch,
                 validation_steps=validation_steps)

Epoch 1/90
358/358 [==============================] - ETA: 0s - loss: 1.5183 - accuracy: 0.4279
Epoch 1: val_accuracy improved from 0.29933 to 0.44154, saving model to final_model_weights.hdf5
358/358 [==============================] - 221s 618ms/step - loss: 1.5183 - accuracy: 0.4279 - val_loss: 1.4728 - val_accuracy: 0.4415
Epoch 2/90
358/358 [==============================] - ETA: 0s - loss: 1.4002 - accuracy: 0.4791
Epoch 2: val_accuracy improved from 0.44154 to 0.50579, saving model to final_model_weights.hdf5
358/358 [==============================] - 218s 610ms/step - loss: 1.4002 - accuracy: 0.4791 - val_loss: 1.3471 - val_accuracy: 0.5058
Epoch 3/90
358/358 [==============================] - ETA: 0s - loss: 1.3139 - accuracy: 0.5151
Epoch 3: val_accuracy did not improve from 0.50579
358/358 [==============================] - 223s 622ms/step - loss: 1.3139 - accuracy: 0.5151 - val_loss: 1.4182 - val_accuracy: 0.4910
Epoch 4/90
358/358 [==============================] - ETA: 0s 

In [126]:
print(f"Test accuracy = {model.evaluate(test_data ,batch_size=test_data.batch_size,steps=test_data.n // test_data.batch_size)[1]*100}%")

112/112 [==============================] - 15s 135ms/step - loss: 1.3757 - accuracy: 0.6366
Test accuracy = 63.65792155265808%


In [131]:
# Code for real time prediction ##

import cv2
import tensorflow as tf
import numpy as np

class_names = ["Angry", "Disgust", "Fear", "Happy", "Neutral", "Sad", "Surprise"]

model = tf.keras.models.load_model('final_model_weights.hdf5')

video = cv2.VideoCapture(1)

faceDetect = cv2.CascadeClassifier('haarcascade_frontalface_default.xml')

while True:
    ret, frame = video.read()
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    faces = faceDetect.detectMultiScale(gray, 1.3, 3)

    for x, y, w, h in faces:
        sub_face_img = gray[y : y + h, x : x + w]
        resized = cv2.resize(sub_face_img, (48, 48))
        normalize = resized / 255.0
        reshaped = np.reshape(normalize, (1, 48, 48, 1))
        result = model.predict(reshaped)
        label = np.argmax(result, axis=1)[0]
        print(label)
        cv2.rectangle(frame, (x, y), (x + w, y + h), (0, 0, 255), 1)
        cv2.rectangle(frame, (x, y), (x + w, y + h), (50, 50, 255), 2)
        cv2.rectangle(frame, (x, y - 40), (x + w, y), (50, 50, 255), -1)
        cv2.putText(frame, class_names[label], (x, y - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.8, (255, 255, 255), 2)
    
    cv2.imshow("Frame", frame)
    k = cv2.waitKey(1)
    if k == ord('q'):
        break

video.release()
cv2.destroyAllWindows()

1/1 [==============================] - 0s 82ms/step
5
1/1 [==============================] - 0s 13ms/step
3
1/1 [==============================] - 0s 13ms/step
0
1/1 [==============================] - 0s 13ms/step
4
1/1 [==============================] - 0s 25ms/step
4
1/1 [==============================] - 0s 13ms/step
4
1/1 [==============================] - 0s 13ms/step
4
1/1 [==============================] - 0s 13ms/step
4
1/1 [==============================] - 0s 13ms/step
3
1/1 [==============================] - 0s 13ms/step
3
1/1 [==============================] - 0s 12ms/step
4
1/1 [==============================] - 0s 12ms/step
4
1/1 [==============================] - 0s 12ms/step
4
1/1 [==============================] - 0s 13ms/step
4
1/1 [==============================] - 0s 12ms/step
3
1/1 [==============================] - 0s 12ms/step
3
1/1 [==============================] - 0s 16ms/step
3
1/1 [==============================] - 0s 14ms/step
3
1/1 [=======================